In [49]:
import pandas as pd
import numpy as np


# Player classification

In [ ]:
from utils import load_merged_impact
player_impact_merged = load_merged_impact()


In [35]:
# Add 2 to all values in batting and bowling impact scores, excluding NaN values
player_impact_merged['Batting_Impact_Score'] = player_impact_merged['Batting_Impact_Score'].apply(lambda x: x + 2 if pd.notna(x) else x)
player_impact_merged['Bowling_Impact_Score'] = player_impact_merged['Bowling_Impact_Score'].apply(lambda x: x + 2 if pd.notna(x) else x)

In [36]:
player_impact_merged['Batting_Impact_Score'].head()


0    1.668987
1    2.327454
2    2.090455
3    2.036906
4    2.241335
Name: Batting_Impact_Score, dtype: float64

In [37]:
player_impact_merged['Bowling_Impact_Score'].head()

0    2.617134
1    1.512578
2    2.110391
3    1.369632
4    2.407865
Name: Bowling_Impact_Score, dtype: float64

In [38]:
player_impact_merged['Batting_Impact_Score'] = player_impact_merged['Batting_Impact_Score'].fillna(0) 
player_impact_merged['Bowling_Impact_Score'] = player_impact_merged['Bowling_Impact_Score'].fillna(0) 


# Step 1: Player Role Classification with Z-scores
def calculate_weighted_impact_zscore(row):
    batting_z = row['Batting_Impact_Score']  # Already a Z-score
    bowling_z = row['Bowling_Impact_Score']  # Already a Z-score
    
    # Determine player role based on statistical significance (1 std dev difference)
    if batting_z > bowling_z + 0.5:  # Batter (significantly better at batting)
        return 0.9 * batting_z + 0.1 * bowling_z, 'Batter'
    elif bowling_z > batting_z + 0.5:  # Bowler (significantly better at bowling)
        return 0.1 * batting_z + 0.9 * bowling_z, 'Bowler'
    elif abs(batting_z - bowling_z) <= 1.5:  # Balanced All-Rounder (skills within 0.5 std dev)
        return 0.55 * batting_z + 0.45 * bowling_z, 'All-Rounder'
    else:  # Mixed Role (some difference but not statistically significant)
        return 0.65 * batting_z + 0.35 * bowling_z, 'Mixed Role'

# Apply the function to create weighted scores and roles
weighted_results = player_impact_merged.apply(
    lambda row: calculate_weighted_impact_zscore(row), axis=1
)

# Split the results into two columns
player_impact_merged['Weighted_Impact_Score'] = weighted_results.apply(lambda x: x[0])
player_impact_merged['Player_Role'] = weighted_results.apply(lambda x: x[1])


## *** player_impact_merged

In [41]:
player_impact_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3138 entries, 0 to 3137
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Player                 3138 non-null   object 
 1   Season                 3138 non-null   int64  
 2   Team                   3138 non-null   object 
 3   total_runs             3138 non-null   int64  
 4   total_balls_faced      3138 non-null   int64  
 5   times_dismissed        3138 non-null   int64  
 6   innings_batted         3138 non-null   int64  
 7   Strike_Rate            3138 non-null   float64
 8   Batting_Average        2474 non-null   float64
 9   z_runs                 3138 non-null   float64
 10  z_avg                  2474 non-null   float64
 11  z_sr                   3138 non-null   float64
 12  Batting_Impact_Score   3138 non-null   float64
 13  total_wickets          3138 non-null   int64  
 14  total_runs_conceded    3138 non-null   int64  
 15  tota

In [42]:

print("Player role distribution with Z-score classification:")
print(player_impact_merged['Player_Role'].value_counts())

Player role distribution with Z-score classification:
Player_Role
Batter         1460
Bowler         1153
All-Rounder     525
Name: count, dtype: int64


In [43]:

# Step 2: Calculate Team Comprehensive Statistics
team_season_stats = player_impact_merged.groupby(['Team', 'Season']).agg(
    total_players=('Player', 'nunique'),
    num_batters=('Player_Role', lambda x: (x == 'Batter').sum()),
    num_bowlers=('Player_Role', lambda x: (x == 'Bowler').sum()),
    num_all_rounders=('Player_Role', lambda x: (x == 'All-Rounder').sum()),
    num_mixed_roles=('Player_Role', lambda x: (x == 'Mixed Role').sum()),
    team_total_impact=('Weighted_Impact_Score', 'sum'),
    team_batting_impact=('Batting_Impact_Score', 'sum'),
    team_bowling_impact=('Bowling_Impact_Score', 'sum')
).reset_index()

# Calculate average impact per player for the team
team_season_stats['avg_player_impact'] = team_season_stats['team_total_impact'] / team_season_stats['total_players']
team_season_stats['avg_batting_impact'] = team_season_stats['team_batting_impact'] / team_season_stats['total_players']
team_season_stats['avg_bowling_impact'] = team_season_stats['team_bowling_impact'] / team_season_stats['total_players']
team_season_stats['all_rounder_ratio'] = team_season_stats['num_all_rounders'] / team_season_stats['total_players']


## *** team_season_stats

In [44]:
team_season_stats.head()

,Team,Season,total_players,num_batters,num_bowlers,num_all_rounders,num_mixed_roles,team_total_impact,team_batting_impact,team_bowling_impact,avg_player_impact,avg_batting_impact,avg_bowling_impact,all_rounder_ratio
0,Chennai Super Kings,2008,19,11,7,1,0,44.871928,38.690953,24.700604,2.361680,2.036366,1.300032,0.052632
1,Chennai Super Kings,2009,18,8,7,3,0,45.304785,38.726447,28.447752,2.516932,2.151469,1.580431,0.166667
2,Chennai Super Kings,2010,23,9,9,5,0,48.268143,36.593630,29.193404,2.098615,1.591027,1.269278,0.217391
3,Chennai Super Kings,2011,17,8,7,2,0,42.190563,31.274173,23.617083,2.481798,1.839657,1.389240,0.117647
4,Chennai Super Kings,2012,17,7,7,3,0,41.191246,30.399621,26.689570,2.423014,1.788213,1.569975,0.176471


In [45]:

# Calculate player distribution statistics for each team-season
team_player_distribution = player_impact_merged.groupby(['Team', 'Season'])['Weighted_Impact_Score'].agg([
    ('min_impact', 'min'),
    ('max_impact', 'max'),
    ('mean_impact', 'mean'),
    ('median_impact', 'median'),
    ('std_impact', 'std'),
    ('q1_impact', lambda x: x.quantile(0.25)),
    ('q3_impact', lambda x: x.quantile(0.75)),
    ('num_players', 'count')
]).reset_index()


In [46]:

# Calculate role-specific distribution stats
role_stats = []
for role in ['Batter', 'Bowler', 'All-Rounder', 'Mixed Role']:
    role_data = player_impact_merged[player_impact_merged['Player_Role'] == role]
    if not role_data.empty:
        role_stats_df = role_data.groupby(['Team', 'Season'])['Weighted_Impact_Score'].agg([
            (f'{role.lower()}_min', 'min'),
            (f'{role.lower()}_max', 'max'),
            (f'{role.lower()}_mean', 'mean'),
            (f'{role.lower()}_std', 'std'),
            (f'{role.lower()}_count', 'count')
        ]).reset_index()
        role_stats.append(role_stats_df)


In [47]:

# Merge all role statistics
team_comprehensive_stats = team_season_stats.merge(
    team_player_distribution,
    on=['Team', 'Season']
)

# Merge role-specific stats
for role_df in role_stats:
    team_comprehensive_stats = team_comprehensive_stats.merge(
        role_df,
        on=['Team', 'Season'],
        how='left'
    )

# Fill NaN values for teams that might not have all roles
role_columns = [col for col in team_comprehensive_stats.columns if any(role in col for role in ['batter', 'bowler', 'all_rounder', 'mixed_role'])]
team_comprehensive_stats[role_columns] = team_comprehensive_stats[role_columns].fillna(0)


In [50]:

# Calculate additional insightful metrics
team_comprehensive_stats['impact_range'] = team_comprehensive_stats['max_impact'] - team_comprehensive_stats['min_impact']
team_comprehensive_stats['batting_bowling_ratio'] = team_comprehensive_stats['team_batting_impact'] / team_comprehensive_stats['team_bowling_impact']
team_comprehensive_stats['impact_consistency'] = 1 / (team_comprehensive_stats['std_impact'] + 0.001)  # Higher = more consistent team
team_comprehensive_stats['role_balance'] = (team_comprehensive_stats['num_batters'] / (team_comprehensive_stats['num_bowlers'] + 0.001)).replace([np.inf, -np.inf], 5)


### ***>>> team_comprehensive_stats <<<

In [51]:
team_comprehensive_stats.head()

,Team,Season,total_players,num_batters,num_bowlers,num_all_rounders,num_mixed_roles,team_total_impact,team_batting_impact,team_bowling_impact,...,bowler_count,all-rounder_min,all-rounder_max,all-rounder_mean,all-rounder_std,all-rounder_count,impact_range,batting_bowling_ratio,impact_consistency,role_balance
0,Chennai Super Kings,2008,19,11,7,1,0,44.871928,38.690953,24.700604,...,7,1.970391,1.970391,1.970391,NaN,1.0,2.845303,1.566397,1.242808,1.571204
1,Chennai Super Kings,2009,18,8,7,3,0,45.304785,38.726447,28.447752,...,7,1.772961,2.045889,1.898785,0.137703,3.0,2.925478,1.361318,1.232350,1.142694
2,Chennai Super Kings,2010,23,9,9,5,0,48.268143,36.593630,29.193404,...,9,0.000000,2.868068,0.777859,1.249339,5.0,4.207215,1.253490,0.831138,0.999889
3,Chennai Super Kings,2011,17,8,7,2,0,42.190563,31.274173,23.617083,...,7,0.000000,2.128030,1.064015,1.504744,2.0,3.599288,1.324218,0.941689,1.142694
4,Chennai Super Kings,2012,17,7,7,3,0,41.191246,30.399621,26.689570,...,7,0.000000,3.413374,1.993740,1.777633,3.0,3.413374,1.139007,1.161681,0.999857


In [64]:

# Add team strength categorization
def categorize_team_strength(row):
    if row['avg_player_impact'] > 2.5:
        return 'Elite'
    elif row['avg_player_impact'] > 2.3:
        return 'Strong'
    elif row['avg_player_impact'] > 2:
        return 'Average'
    else:
        return 'Weak'

team_comprehensive_stats['team_strength'] = team_comprehensive_stats.apply(categorize_team_strength, axis=1)


In [65]:

# Display the results
print("\nTeam Comprehensive Statistics DataFrame shape:", team_comprehensive_stats.shape)
print("\nFirst 10 rows of team statistics:")
pd.set_option('display.max_columns', None)
print(team_comprehensive_stats.head(20).round(3))

# Show top performing teams by average player impact
print("\nTop 10 Teams by Average Player Impact:")
top_teams = team_comprehensive_stats.sort_values('avg_player_impact', ascending=False)
print(top_teams[['Team', 'Season', 'avg_player_impact', 'total_players', 'team_strength']].head(10).round(3))

# Show team strength distribution
print("\nTeam Strength Distribution:")
print(team_comprehensive_stats['team_strength'].value_counts())

# Reset display options
pd.reset_option('display.max_columns')



Team Comprehensive Statistics DataFrame shape: (156, 42)

First 10 rows of team statistics:
                   Team  Season  total_players  num_batters  num_bowlers  \
0   Chennai Super Kings    2008             19           11            7   
1   Chennai Super Kings    2009             18            8            7   
2   Chennai Super Kings    2010             23            9            9   
3   Chennai Super Kings    2011             17            8            7   
4   Chennai Super Kings    2012             17            7            7   
5   Chennai Super Kings    2013             17            7            8   
6   Chennai Super Kings    2014             18            7            9   
7   Chennai Super Kings    2015             14            6            7   
8   Chennai Super Kings    2018             20            9           10   
9   Chennai Super Kings    2019             19            9            9   
10  Chennai Super Kings    2020             20            9           1

In [66]:

# Save the comprehensive team statistics to CSV
team_comprehensive_stats.to_csv('outputs/4_team_season_comprehensive_stats.csv', index=False)
print("\nTeam comprehensive statistics saved to 'team_season_comprehensive_stats_zscore.csv'")



Team comprehensive statistics saved to 'team_season_comprehensive_stats_zscore.csv'


In [67]:

# Also save the player data with weighted scores and roles
player_impact_merged.to_csv('outputs/4_player_impact_with_roles.csv', index=False)
print("Player data with Z-score weighted scores and roles saved to 'player_impact_zscore_with_roles.csv'")


Player data with Z-score weighted scores and roles saved to 'player_impact_zscore_with_roles.csv'


In [68]:

# Display final summary
print("\n=== ANALYSIS COMPLETE ===")
print(f"Total players analyzed: {len(player_impact_merged)}")
print(f"Total team-seasons analyzed: {len(team_comprehensive_stats)}")
print(f"Player roles: {dict(player_impact_merged['Player_Role'].value_counts())}")
print(f"Team strengths: {dict(team_comprehensive_stats['team_strength'].value_counts())}")



=== ANALYSIS COMPLETE ===
Total players analyzed: 3138
Total team-seasons analyzed: 156
Player roles: {'Batter': 1460, 'Bowler': 1153, 'All-Rounder': 525}
Team strengths: {'Average': 94, 'Strong': 35, 'Weak': 19, 'Elite': 8}


# Season-wise rankings

In [61]:
team_comprehensive_stats.head()

,Team,Season,total_players,num_batters,num_bowlers,num_all_rounders,num_mixed_roles,team_total_impact,team_batting_impact,team_bowling_impact,...,all-rounder_min,all-rounder_max,all-rounder_mean,all-rounder_std,all-rounder_count,impact_range,batting_bowling_ratio,impact_consistency,role_balance,team_strength
0,Chennai Super Kings,2008,19,11,7,1,0,44.871928,38.690953,24.700604,...,1.970391,1.970391,1.970391,NaN,1.0,2.845303,1.566397,1.242808,1.571204,Strong
1,Chennai Super Kings,2009,18,8,7,3,0,45.304785,38.726447,28.447752,...,1.772961,2.045889,1.898785,0.137703,3.0,2.925478,1.361318,1.232350,1.142694,Elite
2,Chennai Super Kings,2010,23,9,9,5,0,48.268143,36.593630,29.193404,...,0.000000,2.868068,0.777859,1.249339,5.0,4.207215,1.253490,0.831138,0.999889,Strong
3,Chennai Super Kings,2011,17,8,7,2,0,42.190563,31.274173,23.617083,...,0.000000,2.128030,1.064015,1.504744,2.0,3.599288,1.324218,0.941689,1.142694,Strong
4,Chennai Super Kings,2012,17,7,7,3,0,41.191246,30.399621,26.689570,...,0.000000,3.413374,1.993740,1.777633,3.0,3.413374,1.139007,1.161681,0.999857,Strong


In [77]:
team_comprehensive_stats.columns.tolist()

['Team',
 'Season',
 'total_players',
 'num_batters',
 'num_bowlers',
 'num_all_rounders',
 'num_mixed_roles',
 'team_total_impact',
 'team_batting_impact',
 'team_bowling_impact',
 'avg_player_impact',
 'avg_batting_impact',
 'avg_bowling_impact',
 'all_rounder_ratio',
 'min_impact',
 'max_impact',
 'mean_impact',
 'median_impact',
 'std_impact',
 'q1_impact',
 'q3_impact',
 'num_players',
 'batter_min',
 'batter_max',
 'batter_mean',
 'batter_std',
 'batter_count',
 'bowler_min',
 'bowler_max',
 'bowler_mean',
 'bowler_std',
 'bowler_count',
 'all-rounder_min',
 'all-rounder_max',
 'all-rounder_mean',
 'all-rounder_std',
 'all-rounder_count',
 'impact_range',
 'batting_bowling_ratio',
 'impact_consistency',
 'role_balance',
 'team_strength']

## ***>>> show_season_team_rankings <<<

In [83]:

def show_season_team_rankings(season_year):
    """
    Display score-wise ranking of teams for a specific season
    """
    # Filter for the specific season
    season_data = team_comprehensive_stats[team_comprehensive_stats['Season'] == season_year]
    
    if season_data.empty:
        print(f"No data available for season {season_year}")
        return
    
    # Sort by average player impact (descending)
    ranked_teams = season_data.sort_values('avg_player_impact', ascending=False)
    
    print(f"=== TEAM RANKINGS FOR SEASON {season_year} ===")
    print(f"{'Rank':<4} {'Team':<30} {'Avg Impact':<12} {'Total Impact':<12} {'Min Impact':<12} {'Max Impact':<12} {'Std Impact':<12}{'Impact consistency':<20}  {'Players':<8} ")
    print("-" * 150)
    
    for i, (index, row) in enumerate(ranked_teams.iterrows(), 1):
        print(f"{i:<4} {row['Team']:<30} {row['avg_player_impact']:>10.3f} {row['team_total_impact']:>12.3f}{row['min_impact']:>12.3f}{row['max_impact']:>12.3f}{row['std_impact']:>12.3f}{row['impact_consistency']:>20.3f} "
              f"{row['total_players']:>8} ")
    
    # Additional statistics
    print(f"\nSeason {season_year} Statistics:")
    print(f"Number of teams: {len(ranked_teams)}")
    print(f"Highest average impact: {ranked_teams['avg_player_impact'].max():.3f}")
    print(f"Lowest average impact: {ranked_teams['avg_player_impact'].min():.3f}")
    print(f"Average across all teams: {ranked_teams['avg_player_impact'].mean():.3f}")

# Example usage: Show rankings for 2023 season
show_season_team_rankings(2023)


=== TEAM RANKINGS FOR SEASON 2023 ===
Rank Team                           Avg Impact   Total Impact Min Impact   Max Impact   Std Impact  Impact consistency    Players  
------------------------------------------------------------------------------------------------------------------------------------------------------
1    Gujarat Titans                      2.518       45.332       0.000       4.849       1.203               0.831       18 
2    Chennai Super Kings                 2.445       44.005       0.000       4.016       0.987               1.012       18 
3    Mumbai Indians                      2.221       46.641       0.000       3.876       0.934               1.069       21 
4    Royal Challengers Bangalore         2.200       46.207       0.000       4.293       1.033               0.967       21 
5    Punjab Kings                        2.109       40.068       0.000       2.978       0.829               1.205       19 
6    Lucknow Super Giants                2.099   

In [ ]:

# You can also make it interactive:
try:
    season_input = int(input("\nEnter a season year to view team rankings: "))
    show_season_team_rankings(season_input)
except ValueError:
    print("Please enter a valid year (e.g., 2023)")

# endregion

In [63]:
# region [DETAILED TEAM ANALYSIS FOR SPECIFIC SEASON]

def show_detailed_season_analysis(season_year):
    """
    Show detailed team analysis including role composition for a specific season
    """
    season_data = team_comprehensive_stats[team_comprehensive_stats['Season'] == season_year]
    
    if season_data.empty:
        print(f"No data available for season {season_year}")
        return
    
    # Sort by average impact
    ranked_teams = season_data.sort_values('avg_player_impact', ascending=False)
    
    print(f"=== DETAILED TEAM ANALYSIS FOR SEASON {season_year} ===\n")
    
    for i, (index, row) in enumerate(ranked_teams.iterrows(), 1):
        print(f"{i}. {row['Team']} (Strength: {row['team_strength']})")
        print(f"   Avg Impact: {row['avg_player_impact']:.3f} | Total Impact: {row['team_total_impact']:.3f}")
        print(f"   Players: {row['total_players']} | Batters: {row['num_batters']} | Bowlers: {row['num_bowlers']}")
        print(f"   All-rounders: {row['num_all_rounders']} | Mixed Roles: {row['num_mixed_roles']}")
        print(f"   Consistency: {row['impact_consistency']:.3f} | Role Balance: {row['role_balance']:.2f}\n")

# Example usage
show_detailed_season_analysis(2025)

# endregion

=== DETAILED TEAM ANALYSIS FOR SEASON 2025 ===

1. Royal Challengers Bengaluru (Strength: Strong)
   Avg Impact: 2.410 | Total Impact: 45.787
   Players: 19 | Batters: 9 | Bowlers: 8
   All-rounders: 2 | Mixed Roles: 0
   Consistency: 1.425 | Role Balance: 1.12

2. Gujarat Titans (Strength: Strong)
   Avg Impact: 2.385 | Total Impact: 42.931
   Players: 18 | Batters: 8 | Bowlers: 7
   All-rounders: 3 | Mixed Roles: 0
   Consistency: 0.968 | Role Balance: 1.14

3. Punjab Kings (Strength: Strong)
   Avg Impact: 2.332 | Total Impact: 51.309
   Players: 22 | Batters: 9 | Bowlers: 11
   All-rounders: 2 | Mixed Roles: 0
   Consistency: 1.210 | Role Balance: 0.82

4. Lucknow Super Giants (Strength: Strong)
   Avg Impact: 2.314 | Total Impact: 43.974
   Players: 19 | Batters: 8 | Bowlers: 9
   All-rounders: 2 | Mixed Roles: 0
   Consistency: 1.421 | Role Balance: 0.89

5. Mumbai Indians (Strength: Strong)
   Avg Impact: 2.228 | Total Impact: 49.019
   Players: 22 | Batters: 6 | Bowlers: 10
   

In [71]:

# region [DETAILED TEAM ANALYSIS - FORMATTED]

def show_detailed_season_analysis(season_year):
    """
    Show detailed team analysis including role composition for a specific season
    """
    season_data = team_comprehensive_stats[team_comprehensive_stats['Season'] == season_year]
    
    if season_data.empty:
        print(f"No data available for season {season_year}")
        return
    
    # Sort by average impact
    ranked_teams = season_data.sort_values('avg_player_impact', ascending=False)
    
    print(f"╔{'═'*85}╗")
    print(f"║{'DETAILED TEAM ANALYSIS FOR SEASON ' + str(season_year):^85}║")
    print(f"╚{'═'*85}╝\n")
    
    for i, (index, row) in enumerate(ranked_teams.iterrows(), 1):
        print(f"🏆 #{i:2d} {row['Team']:15} ({row['team_strength']} Team)")
        print(f"   ┌{'─'*40}")
        print(f"   │ Impact:    {row['avg_player_impact']:7.3f} (avg)  {row['team_total_impact']:9.3f} (total)")
        print(f"   │ Players:   {row['total_players']:3d} total")
        print(f"   │ Composition: {row['num_batters']:2d} batters, {row['num_bowlers']:2d} bowlers")
        print(f"   │             {row['num_all_rounders']:2d} all-rounders, {row['num_mixed_roles']:2d} mixed roles")
        print(f"   │ Balance:   {row['role_balance']:5.2f} ratio, {row['impact_consistency']:6.3f} consistency")
        print(f"   └{'─'*40}\n")

# Example usage
show_detailed_season_analysis(2025)

# endregion


╔═════════════════════════════════════════════════════════════════════════════════════╗
║                       DETAILED TEAM ANALYSIS FOR SEASON 2025                        ║
╚═════════════════════════════════════════════════════════════════════════════════════╝

🏆 # 1 Royal Challengers Bengaluru (Strong Team)
   ┌────────────────────────────────────────
   │ Impact:      2.410 (avg)     45.787 (total)
   │ Players:    19 total
   │ Composition:  9 batters,  8 bowlers
   │              2 all-rounders,  0 mixed roles
   │ Balance:    1.12 ratio,  1.425 consistency
   └────────────────────────────────────────

🏆 # 2 Gujarat Titans  (Strong Team)
   ┌────────────────────────────────────────
   │ Impact:      2.385 (avg)     42.931 (total)
   │ Players:    18 total
   │ Composition:  8 batters,  7 bowlers
   │              3 all-rounders,  0 mixed roles
   │ Balance:    1.14 ratio,  0.968 consistency
   └────────────────────────────────────────

🏆 # 3 Punjab Kings    (Strong Team)
   ┌───

In [72]:

# region [INTERACTIVE SEASON SELECTOR]

def interactive_season_selector():
    """Interactive function to select and view any season"""
    available_seasons = sorted(team_comprehensive_stats['Season'].unique())
    
    print("📅 Available seasons:", available_seasons)
    
    try:
        season_input = int(input("\nEnter a season year to view team rankings: "))
        if season_input in available_seasons:
            print("\n" + "="*80)
            show_season_team_rankings(season_input)
            print("\n" + "="*80)
            show_detailed_season_analysis(season_input)
        else:
            print(f"❌ Season {season_input} not found in available data.")
            print(f"   Available seasons: {available_seasons}")
    except ValueError:
        print("❌ Please enter a valid year (e.g., 2023)")

# Uncomment to use interactive mode
# interactive_season_selector()

# endregion